# Task 3: Deploy agents on Amazon Bedrock AgentCore

In this task, you will transition from local development to production deployment using Amazon Bedrock AgentCore. You will take the sophisticated multi-agent financial advisory system built in Task 1 & Task 2 and deploy it to AWS's enterprise-grade agent hosting platform - AgentCore Runtime.

![architecture](./images/architecture.png)


## What You Will Learn

You will learn how to leverage AgentCore's purpose-built infrastructure for running agents at scale while maintaining security, performance, and reliability standards required for enterprise applications.

## Amazon Bedrock AgentCore Runtime

[Amazon Bedrock AgentCore Runtime](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/agents-tools-runtime.html) provides a secure, serverless, and purpose-built hosting environment for deploying and running AI agents or tools, shortening the time to value from experiments to production-grade agents.

[Learn more about how AgentCore Runtime works](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/runtime-how-it-works.html)

## Amazon Bedrock AgentCore Observability

[AgentCore Observability](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/observability.html) helps you trace, debug, and monitor agent performance in production environments. It offers detailed visualizations of each step in the agent workflow, enabling you to inspect an agent's execution path, audit intermediate outputs, and debug performance bottlenecks and failures.

In [2]:
%%capture
# Install dependencies for AgentCore deployment
# !pip install --force-reinstall -U -r requirements.txt --quiet --disable-pip-version-check
!pip install -U -r requirements.txt --quiet --disable-pip-version-check

In [3]:
# Import AgentCore Runtime deployment tools and utilities
import uuid
from utils import setup_cognito_user_pool, reauthenticate_user
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
from typing import Any, Optional
import urllib.parse
import requests
import json

In [4]:
# Initialize AWS session and get current region
boto_session = Session()
region = boto_session.region_name

### Task 3.1: Preparing your agent for deployment on AgentCore Runtime

Let's now deploy our agents to `AgentCore Runtime`. To do so we need to:

- Import the Runtime App with `from bedrock_agentcore.runtime import BedrockAgentCoreApp`
- Initialize the App in our code with `app = BedrockAgentCoreApp()`
- Decorate the invocation function with the `@app.entrypoint` decorator
- Let AgentCore Runtime control the running of the agent with `app.run()`

In [6]:
%%writefile main.py
# Create AgentCore-compatible deployment file with streaming endpoint and memory integration

from strands import Agent, tool
from strands.models import BedrockModel
from strands.agent.conversation_manager import SummarizingConversationManager

from budget_agent import FinancialReport, budget_agent
from financial_analysis_agent import financial_analysis_agent
from bedrock_agentcore import BedrockAgentCoreApp
from bedrock_agentcore.memory import MemoryClient

from utils import get_guardrail_id
import uuid
import os
import logging

# Configure logging for error tracking and debugging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Initialize AgentCore app
app = BedrockAgentCoreApp()

ORCHESTRATOR_PROMPT = """You are a comprehensive financial advisor orchestrator that coordinates between specialized financial agents to provide complete financial guidance. 

Your specialized agents are:
1. **budget_agent**: Handles budgeting, spending analysis, savings recommendations, and expense tracking
2. **financial_analysis_agent_tool**: Handles investment analysis, stock research, portfolio creation, and performance comparisons

Guidelines for using your agents:
- Use **budget_agent** for questions about: budgets, spending habits, expense tracking, savings goals, debt management
- Use **financial_analysis_agent_tool** for questions about: stocks, investments, portfolios, market analysis, investment recommendations
- You can use both agents together for comprehensive financial planning
- Always provide a cohesive summary that combines insights from multiple agents when applicable
- Maintain a helpful, professional tone and include appropriate disclaimers about financial advice

When a user asks a question:
1. Determine which agent(s) are most appropriate
2. Call the relevant agent(s) with focused queries
3. Synthesize the responses into a coherent, comprehensive answer
4. Provide actionable next steps when possible"""

# Add conversation management to maintain context
conversation_manager = SummarizingConversationManager(
    summary_ratio=0.3,  # Summarize 30% of messages when context reduction is needed
    preserve_recent_messages=5,  # Always keep 5 most recent messages
)

# Configure Bedrock model
bedrock_model = BedrockModel(
    model_id="global.anthropic.claude-haiku-4-5-20251001-v1:0",
    region_name="us-west-2",
    temperature=0.0,  # Deterministic responses for financial advice
    guardrail_id=get_guardrail_id(),
    guardrail_version="DRAFT",
    guardrail_trace="enabled",
)


@tool
def budget_agent_tool(query: str) -> FinancialReport:
    """Generate structured financial reports with budget analysis and recommendations."""
    logger.info(f"budget_agent_tool called with query: {query[:100]}...")
    try:
        structured_response = budget_agent.structured_output(
            output_model=FinancialReport, prompt=query
        )
        logger.info("budget_agent_tool completed successfully")
        return structured_response
    except Exception as e:
        logger.error(f"Error in budget_agent_tool: {e}", exc_info=True)
        # Return a default structured response on error
        return FinancialReport(
            monthly_income=0.0,
            budget_categories=[],
            recommendations=[f"Error generating report: {str(e)}"],
            financial_health_score=1,
        )


@tool
def financial_analysis_agent_tool(query: str) -> str:
    """Handle investment analysis queries including stock research, portfolio creation, and performance comparisons."""
    logger.info(f"financial_analysis_agent_tool called with query: {query[:100]}...")
    try:
        response = financial_analysis_agent(query)
        logger.info("financial_analysis_agent_tool completed successfully")
        return str(response)
    except Exception as e:
        logger.error(f"Error in financial_analysis_agent_tool: {e}", exc_info=True)
        return f"❌ Financial analysis error: {str(e)}"


# Initialize orchestrator agent at module level
logger.info("Initializing orchestrator agent...")
orchestrator_agent = Agent(
    model=bedrock_model,
    system_prompt=ORCHESTRATOR_PROMPT,
    tools=[budget_agent_tool, financial_analysis_agent_tool],
    conversation_manager=conversation_manager,
)
logger.info("Orchestrator agent initialized successfully")


@app.entrypoint
async def invoke(payload):
    """Your AI agent function with memory integration"""
    user_message = payload["prompt"]
    actor_id = payload.get("actor_id", "default_user")  # User identifier
    session_id = payload.get("session_id", str(uuid.uuid4()))  # Session identifier
    
    # Retrieve relevant memories before processing
    memory_id = os.environ.get('AGENTCORE_MEMORY_ID')
    if memory_id:
        try:
            memory_client = MemoryClient()
            # Query relevant memories for context
            relevant_memories = memory_client.query_records(
                memory_id=memory_id,
                query=user_message,
                namespace=f"finance/user/{actor_id}/facts",
                max_results=5
            )
            
            # Add memory context to system prompt if available
            if relevant_memories:
                memory_context = "\n\nRelevant user information from previous conversations:\n"
                for record in relevant_memories:
                    memory_context += f"- {record.get('content', '')}\n"
                
                # Enhance agent with memory context
                enhanced_prompt = ORCHESTRATOR_PROMPT + memory_context
                orchestrator_agent.system_prompt = enhanced_prompt
        except Exception as e:
            logger.warning(f"Error retrieving memories: {e}")
    
    # Stream response
    full_response = ""
    async for event in orchestrator_agent.stream_async(user_message):
        if "data" in event:
            chunk = event["data"]
            full_response += chunk
            yield chunk
    
    # Store conversation in memory after response
    if memory_id:
        try:
            memory_client.create_event(
                memory_id=memory_id,
                actor_id=actor_id,
                session_id=session_id,
                content=f"User: {user_message}\nAssistant: {full_response}"
            )
        except Exception as e:
            logger.warning(f"Error storing memory: {e}")


if __name__ == "__main__":
    app.run()

Overwriting main.py


## What happens behind the scenes?

When you use `BedrockAgentCoreApp`, it automatically:

* Creates an HTTP server that listens on port 8080
* Implements the required `/invocations` endpoint for processing the agent's requirements
* Implements the `/ping` endpoint for health checks (very important for asynchronous agents)
* Handles proper content types and response formats
* Manages error handling according to AWS standards

### Task 3.2: Setting up Amazon Cognito for Authentication

AgentCore Runtime requires authentication. We'll use Amazon Cognito to provide JWT tokens for accessing our deployed agent server.

In [7]:
# Set up Amazon Cognito for AgentCore Runtime authentication
print("Setting up Amazon Cognito user pool...")

cognito_config = setup_cognito_user_pool()

print("Cognito setup completed ✓")
print(f"User Pool ID: {cognito_config.get('user_pool_id', 'N/A')}")
print(f"Client ID: {cognito_config.get('client_id', 'N/A')}")

Setting up Amazon Cognito user pool...
✓ Credentials stored in AWS Secrets Manager: agentcore-lab-credentials-11746c3c

⚠️  IMPORTANT: SAVE THESE CREDENTIALS NOW!
Username: labuser-decd0121
Password: 2o0tnSffetutcx#I
Secret Name: agentcore-lab-credentials-11746c3c
Pool ID: us-west-2_G7mI5C1LN
Discovery URL: https://cognito-idp.us-west-2.amazonaws.com/us-west-2_G7mI5C1LN/.well-known/openid-configuration
Client ID: 12hhu3bfefctnr9c3u7nbcaka6
Bearer Token: eyJraWQiOiJpVkFiRFZ3dmdTVVJjamE3YmhieVwvZ2JoS2p1MXVuMHhLZFlFM1lWWVZsRT0iLCJhbGciOiJSUzI1NiJ9.eyJzdWIiOiI2ODExYjNmMC00MDQxLTcwNmUtN2Q3Zi0yZDlmZDgxYTkzZmMiLCJpc3MiOiJodHRwczpcL1wvY29nbml0by1pZHAudXMtd2VzdC0yLmFtYXpvbmF3cy5jb21cL3VzLXdlc3QtMl9HN21JNUMxTE4iLCJjbGllbnRfaWQiOiIxMmhodTNiZmVmY3RucjljM3U3bmJjYWthNiIsIm9yaWdpbl9qdGkiOiJjZWJhZWIyMy04ODllLTQzY2ItYTRlZC1lMWM1YmQ3MGUwZWIiLCJldmVudF9pZCI6ImIzNTc3MWVmLTAwNWMtNDM1Zi04NDNkLWZkY2FjNmVlYzBiOSIsInRva2VuX3VzZSI6ImFjY2VzcyIsInNjb3BlIjoiYXdzLmNvZ25pdG8uc2lnbmluLnVzZXIuYWRtaW4iLCJhdXRoX3RpbWUiO

In [8]:
# Configure JWT authorization for AgentCore Runtime
auth_config = {
    "customJWTAuthorizer": {
        "allowedClients": [cognito_config["client_id"]],
        "discoveryUrl": cognito_config["discovery_url"],
    }
}

### Task 3.2.1: Configure AgentCore Memory

AgentCore Memory enables your agent to remember context across conversations. We'll configure three memory strategies:

1. **Semantic Strategy**: Extracts and stores factual information (income, expenses, goals)
2. **User Preference Strategy**: Captures user preferences (risk tolerance, investment style)
3. **Summary Strategy**: Creates conversation summaries for context

This allows the financial advisor to provide personalized advice based on previous interactions.

In [9]:
# Import AgentCore Memory Client
from bedrock_agentcore.memory import MemoryClient
from bedrock_agentcore.memory.constants import StrategyType


# Define memory strategies for financial advisor
memory_strategies = [
    {
        StrategyType.SEMANTIC.value: {
            "name": "financial_facts",
            "description": "Extracts and stores user's financial facts (income, expenses, goals)",
            "namespaces": ["finance/user/{actorId}/facts"]
        }
    },
    {
        StrategyType.USER_PREFERENCE.value: {
            "name": "user_preferences",
            "description": "Captures user preferences for investment risk, budget priorities",
            "namespaces": ["finance/user/{actorId}/preferences"]
        }
    },
    {
        StrategyType.SUMMARY.value: {
            "name": "conversation_summary",
            "description": "Summarizes financial advisory conversations",
            "namespaces": ["finance/user/{actorId}/sessions/{sessionId}"]
        }
    }
]

In [10]:
# Initialize Memory Client and create memory resource
# Note that creating memory resources can take several minutes
memory_client = MemoryClient()

print("Creating AgentCore Memory for financial advisor...")
try:
    # Check if memory already exists
    existing_memories = memory_client.list_memories()
    memory_id = None
    
    for mem in existing_memories:
        if mem.get('name') == 'FinancialAdvisorMemory':
            memory_id = mem['id']
            print(f"✓ Using existing memory: {memory_id}")
            break
    
    if not memory_id:
        memory = memory_client.create_memory_and_wait(
            name="FinancialAdvisorMemory",
            strategies=memory_strategies,
            description="Memory for personal finance advisor agent",
            event_expiry_days=90  # Keep financial data for 90 days
        )
        memory_id = memory["id"]
        print(f"✓ Memory created: {memory_id}")
        print(f"  - Semantic strategy: Extracts financial facts")
        print(f"  - User preference strategy: Captures preferences")
        print(f"  - Summary strategy: Creates conversation summaries")
    
except Exception as e:
    print(f"❌ Error setting up memory: {e}")
    print("Continuing without memory integration...")
    memory_id = None

Creating AgentCore Memory for financial advisor...
✓ Memory created: FinancialAdvisorMemory-segF4298qr
  - Semantic strategy: Extracts financial facts
  - User preference strategy: Captures preferences
  - Summary strategy: Creates conversation summaries


### Task 3.3: Deploying the agent to AgentCore Runtime

The `CreateAgentRuntime` operation supports comprehensive configuration options, letting you specify container images, environment variables, and encryption settings. You can also configure protocol settings (HTTP, MCP) and authorization mechanisms to control how your clients communicate with the agent. 

**Note:** Operations best practice is to package code as a container and push to ECR using CI/CD pipelines and IaC.

In this tutorial, we will use the Amazon Bedrock AgentCore Python SDK to easily package your artifacts and deploy them to AgentCore Runtime.

### Task 3.3.1: Configure AgentCore Runtime deployment

First, we will use our starter toolkit to configure the AgentCore Runtime deployment with an entrypoint, the execution role we just created, and a requirements file. We will also configure the starter kit to auto-create the Amazon ECR repository on launch.

During the configure step, your Dockerfile will be generated based on your application code.

![runtime](./images/runtime_overview.png)

In [11]:
# Configure AgentCore Runtime deployment settings
agentcore_runtime = Runtime()

agent_name = "personal_finance_agent"

print("Configuring AgentCore Runtime...")
response = agentcore_runtime.configure(
    entrypoint="main.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name,
    authorizer_configuration=auth_config,
)

print("Configuration completed ✓")

# Note: You may see a platform mismatch warning (linux/amd64 vs linux/arm64).
# This is expected and can be safely ignored. AgentCore Runtime uses ARM64/Graviton
# processors, and AWS will automatically rebuild the image for the correct architecture
# during deployment. All dependencies in this lab are compatible with ARM64.

Entrypoint parsed: file=C:\Users\kenkr\git\aws-teaching-demos\bedrock-agentcore-lab-demo\finance-personal-assistant\main.py, bedrock_agentcore_name=main
Configuring BedrockAgentCore agent: personal_finance_agent


Configuring AgentCore Runtime...


⚠️  [WARNING] Platform mismatch: Current system is 'linux/amd64' but Bedrock AgentCore requires 'linux/arm64'.
For deployment options and workarounds, see: 
https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/getting-started-custom.html

Generated .dockerignore
Generated Dockerfile: c:\Users\kenkr\git\aws-teaching-demos\bedrock-agentcore-lab-demo\finance-personal-assistant\Dockerfile
Generated .dockerignore: c:\Users\kenkr\git\aws-teaching-demos\bedrock-agentcore-lab-demo\finance-personal-assistant\.dockerignore
Setting 'personal_finance_agent' as default agent
Bedrock AgentCore configured: c:\Users\kenkr\git\aws-teaching-demos\bedrock-agentcore-lab-demo\finance-personal-assistant\.bedrock_agentcore.yaml


Configuration completed ✓


**Note:** You may see a platform mismatch warning during configuration:

```
⚠️  [WARNING] Platform mismatch: Current system is 'linux/amd64' but Bedrock AgentCore requires 'linux/arm64'.
```

**This warning can be safely ignored.** AgentCore Runtime uses AWS Graviton (ARM64) processors for better performance and cost efficiency. AWS will automatically rebuild your Docker image for the correct ARM64 architecture during the deployment process. All Python dependencies used in this lab are fully compatible with ARM64.

### Task 3.3.2: Launching agent to AgentCore Runtime

Now that we have a Dockerfile, let's launch the agent to the AgentCore Runtime. This will create the Amazon ECR repository and the AgentCore Runtime.

In [12]:
# Deploy agent to AgentCore Runtime (creates ECR repo and runtime)
print("Launching Agent server to AgentCore Runtime...")
print("This may take several minutes...")

launch_result = agentcore_runtime.launch(
    env_vars={
        "OTEL_PYTHON_EXCLUDED_URLS": "/ping,/invocations",
        "AGENTCORE_MEMORY_ID": memory_id if memory_id else ""
    }
)

print("Launch completed ✓")
print(f"Agent ARN: {launch_result.agent_arn}")
print(f"Agent ID: {launch_result.agent_id}")

🚀 CodeBuild mode: building in cloud (RECOMMENDED - DEFAULT)
   • Build ARM64 containers in the cloud with CodeBuild
   • No local Docker required
💡 Available deployment modes:
   • runtime.launch()                           → CodeBuild (current)
   • runtime.launch(local=True)                 → Local development
   • runtime.launch(local_build=True)           → Local build + cloud deploy (NEW)
Starting CodeBuild ARM64 deployment for agent 'personal_finance_agent' to account 011673140073 (us-west-2)
Setting up AWS resources (ECR repository, execution roles)...
Getting or creating ECR repository for agent: personal_finance_agent


Launching Agent server to AgentCore Runtime...
This may take several minutes...
Repository doesn't exist, creating new ECR repository: bedrock-agentcore-personal_finance_agent


✅ ECR repository available: 011673140073.dkr.ecr.us-west-2.amazonaws.com/bedrock-agentcore-personal_finance_agent
Getting or creating execution role for agent: personal_finance_agent
Using AWS region: us-west-2, account ID: 011673140073
Role name: AmazonBedrockAgentCoreSDKRuntime-us-west-2-5d12c2867b
Role doesn't exist, creating new execution role: AmazonBedrockAgentCoreSDKRuntime-us-west-2-5d12c2867b
Starting execution role creation process for agent: personal_finance_agent
✓ Role creating: AmazonBedrockAgentCoreSDKRuntime-us-west-2-5d12c2867b
Creating IAM role: AmazonBedrockAgentCoreSDKRuntime-us-west-2-5d12c2867b
✓ Role created: arn:aws:iam::011673140073:role/AmazonBedrockAgentCoreSDKRuntime-us-west-2-5d12c2867b
✓ Execution policy attached: BedrockAgentCoreRuntimeExecutionPolicy-personal_finance_agent
Role creation complete and ready for use with Bedrock AgentCore
✅ Execution role available: arn:aws:iam::011673140073:role/AmazonBedrockAgentCoreSDKRuntime-us-west-2-5d12c2867b
Prepari

Launch completed ✓
Agent ARN: arn:aws:bedrock-agentcore:us-west-2:011673140073:runtime/personal_finance_agent-uYgRti2HCd
Agent ID: personal_finance_agent-uYgRti2HCd


#### What Just Happened?

The `agentcore_runtime.launch()` command performed several important operations:

1. **Created an Amazon ECR Repository**: A private container registry to store your agent's Docker image
2. **Built the Docker Image**: Packaged your agent code, dependencies, and configuration into a container
3. **Pushed to ECR**: Uploaded the container image to your ECR repository
4. **Created AgentCore Runtime**: Deployed a serverless runtime environment that will execute your agent
5. **Configured Environment Variables**: Set up memory integration and observability settings

Let's verify what was created by querying AWS services:

In [13]:
import boto3
import json

# Initialize AWS clients
ecr_client = boto3.client('ecr')
agentcore_client = boto3.client('bedrock-agentcore-control')

print("=" * 80)
print("DEPLOYED RESOURCES VERIFICATION")
print("=" * 80)

# 1. Verify ECR Repository
print("\n📦 Amazon ECR Repository:")
print("-" * 80)
try:
    # Extract repository name from agent ARN or use a pattern
    # AgentCore typically creates repos with a specific naming pattern
    repos = ecr_client.describe_repositories(maxResults=10)
    
    # Find the most recently created repository (likely our agent)
    if repos['repositories']:
        latest_repo = sorted(repos['repositories'], 
                           key=lambda x: x['createdAt'], 
                           reverse=True)[0]
        
        print(f"Repository Name: {latest_repo['repositoryName']}")
        print(f"Repository URI: {latest_repo['repositoryUri']}")
        print(f"Created: {latest_repo['createdAt']}")
        
        # Get image details
        images = ecr_client.describe_images(
            repositoryName=latest_repo['repositoryName'],
            maxResults=5
        )
        print(f"\nContainer Images: {len(images['imageDetails'])} image(s)")
        for img in images['imageDetails'][:3]:  # Show up to 3 most recent
            tags = img.get('imageTags', ['<untagged>'])
            size_mb = img['imageSizeInBytes'] / (1024 * 1024)
            print(f"  • Tag: {tags[0]}, Size: {size_mb:.1f} MB, Pushed: {img['imagePushedAt']}")
    else:
        print("No ECR repositories found")
except Exception as e:
    print(f"Error retrieving ECR info: {e}")

# 2. Verify AgentCore Runtime
print("\n🤖 AgentCore Runtime:")
print("-" * 80)
try:
    # List all runtimes and find ours
    runtime_response = agentcore_client.list_agent_runtimes(maxResults=10)
    
    # Find our runtime by matching the agent_id from launch_result
    our_runtime = None
    for runtime in runtime_response.get('agentRuntimes', []):
        if runtime['agentRuntimeId'] == launch_result.agent_id:
            our_runtime = runtime
            break
    
    if our_runtime:
        print(f"Runtime Name: {our_runtime.get('agentRuntimeName', 'N/A')}")
        print(f"Runtime ARN: {our_runtime['agentRuntimeArn']}")
        print(f"Runtime ID: {our_runtime['agentRuntimeId']}")
        print(f"Status: {our_runtime.get('status', 'N/A')}")
        print(f"Version: {our_runtime.get('agentRuntimeVersion', 'N/A')}")
        print(f"Last Updated: {our_runtime.get('lastUpdatedAt', 'N/A')}")
        if our_runtime.get('description'):
            print(f"Description: {our_runtime['description']}")
    else:
        print(f"Runtime not found in list. Showing launch details:")
        print(f"  • Agent ARN: {launch_result.agent_arn}")
        print(f"  • Agent ID: {launch_result.agent_id}")
        print(f"\nNote: Runtime may still be initializing. Total runtimes found: {len(runtime_response.get('agentRuntimes', []))}")
            
except Exception as e:
    print(f"Error retrieving AgentCore Runtime info: {e}")
    print(f"\nShowing runtime details from launch:")
    print(f"  • Agent ARN: {launch_result.agent_arn}")
    print(f"  • Agent ID: {launch_result.agent_id}")

print("\n" + "=" * 80)
print("✅ Verification Complete")
print("=" * 80)
print("\nYour agent is now deployed and ready to be invoked!")

DEPLOYED RESOURCES VERIFICATION

📦 Amazon ECR Repository:
--------------------------------------------------------------------------------
Repository Name: bedrock-agentcore-personal_finance_agent
Repository URI: 011673140073.dkr.ecr.us-west-2.amazonaws.com/bedrock-agentcore-personal_finance_agent
Created: 2026-02-18 11:03:16.523000-05:00

Container Images: 1 image(s)
  • Tag: latest, Size: 233.8 MB, Pushed: 2026-02-18 11:04:26.910000-05:00

🤖 AgentCore Runtime:
--------------------------------------------------------------------------------
Runtime Name: personal_finance_agent
Runtime ARN: arn:aws:bedrock-agentcore:us-west-2:011673140073:runtime/personal_finance_agent-uYgRti2HCd
Runtime ID: personal_finance_agent-uYgRti2HCd
Status: READY
Version: 1
Last Updated: 2026-02-18 16:04:31.241616+00:00

✅ Verification Complete

Your agent is now deployed and ready to be invoked!


### Task 3.4: Invoking AgentCore Runtime

Finally, we can invoke our AgentCore Runtime with a payload

In [14]:
# Authenticate user and get bearer token for API access
bearer_token = reauthenticate_user(
    client_id=cognito_config["client_id"],
    secret_name=cognito_config["secret_name"]
)

✓ Retrieved credentials from Secrets Manager: agentcore-lab-credentials-11746c3c
✓ Successfully reauthenticated user


In [15]:
def invoke_endpoint(
    agent_arn: str,
    payload,
    session_id: str,
    bearer_token: Optional[str],
    region: str = "us-west-2",
    endpoint_name: str = "DEFAULT",
) -> Any:
    """Invoke agent endpoint using HTTP request with bearer token."""
    escaped_arn = urllib.parse.quote(agent_arn, safe="")
    url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{escaped_arn}/invocations"
    headers = {
        "Authorization": f"Bearer {bearer_token}",
        "Content-Type": "application/json",
        "X-Amzn-Bedrock-AgentCore-Runtime-Session-Id": session_id,
    }

    try:
        body = json.loads(payload) if isinstance(payload, str) else payload
    except json.JSONDecodeError:
        body = {"payload": payload}

    try:
        # This is the actual invocation of the agent endpoint. 
        response = requests.post(
            url,
            params={"qualifier": endpoint_name},
            headers=headers,
            json=body,
            timeout=100,
            stream=True,
        )
        last_data = False
        for line in response.iter_lines(chunk_size=1):
            if line:
                line = line.decode("utf-8")
                if line.startswith("data: "):
                    last_data = True
                    line = line[6:]
                    line = line.replace('"', "")
                    yield line
                elif line:
                    line = line.replace('"', "")
                    if last_data:
                        yield "\n" + line
                    last_data = False

    except requests.exceptions.RequestException as e:
        print("Failed to invoke agent endpoint: %s", str(e))
        raise

In [16]:
for chunk in invoke_endpoint(
    agent_arn=launch_result.agent_arn,
    payload={
        "prompt": "I make $6000/month and want to start investing $500/month. Help me create a budget and suggest an investment portfolio.",
        "actor_id": "test_user_123"
    },
    session_id=str(uuid.uuid4()),
    bearer_token=bearer_token,
):
    print(chunk.replace("\\n", "\n"), end="")

I'll help you create a comprehensive financial plan by developing a budget and suggesting an investment portfolio. Let me gather insights from both our budgeting and investment specialists.Perfect! I've created a comprehensive financial plan for you. Here's your complete roadmap:

## **Your Monthly Budget ($6,000 Income)**

| Category | Amount | % of Income |
|----------|--------|------------|
| **Housing** | $1,800 | 30% |
| **Transportation** | $600 | 10% |
| **Food & Groceries** | $480 | 8% |
| **Utilities & Phone** | $300 | 5% |
| **Insurance** | $240 | 4% |
| **Investment** | $500 | 8.33% |
| **Emergency Fund** | $300 | 5% |
| **Entertainment & Dining** | $360 | 6% |
| **Personal Care & Clothing** | $180 | 3% |
| **Miscellaneous Buffer** | $240 | 4% |
| **TOTAL** | $6,000 | 100% |

**Financial Health Score: 8/10** ✅

---

## **Your Investment Portfolio ($500/Month)**

A balanced, beginner-friendly portfolio with established companies:

- **AAPL** (Apple) - $125/month - Technology 

**Task complete:** You have successfully deployed your multi-agent system to production using Amazon Bedrock AgentCore with memory integration. Your financial assistant now runs in a fully managed, enterprise-grade environment with:

- **AgentCore Runtime**: Auto-scaling, observability, and security built in
- **AgentCore Memory**: Context retention across conversations using semantic, user preference, and summary strategies
- **Production-Ready Architecture**: Guardrails, authentication, and error handling

The memory integration enables your agent to remember user financial information, preferences, and conversation history, providing personalized advice across sessions. You have completed the full journey from development to production deployment of agentic AI applications on AWS.

### Next Steps

You have completed this notebook. To continue to the next part of the lab, return to the lab instructions and continue with **Cleanup**.